In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = 'transcrittomic'

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
adata = sc.read(DATA_PATH)
adata

c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 250920 × 37944
    obs: 'dataset', 'disease', 'cell_type', 'sample', 'patient_id', 'time', 'cell_types_labels', 'patient', 'MS/HC'
    var: 'gene_ids', 'feature_types'

In [3]:
adata.obs.head()

,dataset,disease,cell_type,sample,patient_id,time,cell_types_labels,patient,MS/HC
AAACCCAAGACTGTTC-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells,NaN,MS
AAACCCAAGGATCATA-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells,NaN,MS
AAACCCAGTTATTCTC-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,B cells,NaN,MS
AAACCCATCATGAGGG-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells,NaN,MS
AAACGAAAGCCAGTAG-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,B cells,NaN,MS


In [4]:
adata.obs.describe()

,dataset,disease,cell_type,sample,patient_id,time,cell_types_labels,patient,MS/HC
count,250920,250920,250920,140355,169641,72317,250920,178603,250920
unique,3,6,2,39,16,2,19,23,2
top,GSE194078,MS,PBMC,GSM4104138,KSH,M3,T cells,GSM5827380,MS
freq,110565,151730,177189,7819,22578,37203,193689,22578,151730


In [5]:
adata.X

<250920x37944 sparse matrix of type '<class 'numpy.float64'>'
	with 385236280 stored elements in Compressed Sparse Row format>

In [6]:
adata.var.describe()

,gene_ids,feature_types
count,37944,37944
unique,37944,1
top,ENSG00000243485,Gene Expression
freq,1,37944


### Preproc

In [7]:
sc.pl.highest_expr_genes(adata, n_top=20)

MemoryError: Unable to allocate 1.44 GiB for an array with shape (385236280,) and data type int32

In [ ]:
# basic filtering
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
mito_genes = adata.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

In [ ]:
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'],
    jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='percent_mito')
sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

Identify highly-variable genes.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata = adata[:, adata.var.highly_variable] # filtering

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

In [ ]:
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

Scale each gene to unit variance. Clip values exceeding standard deviation 10. 

In [ ]:
sc.pp.scale(adata, max_value=10)

### PCA

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color='CST3')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

### Save results

In [ ]:
adata

In [ ]:
adata.write('write/transcrittomic_preproc.h5ad')

### Plots

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
ax = plt.gca()
adata.obs['cell_types_labels'].value_counts().plot(kind='bar', ax=ax)
total= len(adata.obs['cell_types_labels'])
for p in ax.patches:
    percentage = '{:.4f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width() / 2 - 0.1
    y = p.get_y() + p.get_height() + 10
    plt.annotate(percentage, (x, y), rotation=90)
plt.xticks(rotation=90)
plt.yscale('log')
plt.title('Predicted labels counts, dataset GSE194078')
plt.xlabel('Annotation')
plt.ylabel('Count')
plt.tight_layout()
plt.show()